In [10]:
import pandas as pd
import yfinance as yf

In [11]:
def get_OHLC_data(tickers:list, exit_etf:str, start_date:str, end_date:str):
    # Initialize empty Dataframe
    data = pd.DataFrame()

    # Download all ticker data and concatenate them to the dataframe
    for ticker in tickers:
        data2 = yf.download(ticker, start=f'{start_date}-01-01', end=f'{end_date}-01-01', interval='1d', progress=False)
        data2['Ticker'] = ticker
        data2['exit'] = exit_etf
        data = pd.concat([data, data2])

    # Download the exit ticker data and merge it to each row
    data2 = yf.download(exit_etf, start=f'{start_date}-01-01', end=f'{end_date}-01-01', interval='1d', progress=False)
    data, data2 = data.reset_index(), data2.reset_index()
    data = data.merge(data2, on='Date', suffixes=('', f'_{exit_etf}'))

    # Sort by etf, then by date
    data = data.sort_values(by=['Ticker', 'Date'])
    data = data.reset_index(drop=True)
    
    return data

In [12]:
def total_returns(group, exit_etf:str):

    # Find the day we would've triggered the stoploss and sold in the period
    sell_index = group.index[group['Sell?']].min()

    # If it did trigger:
    if group['Sell?'].any():

        # Find the etf return when selling on that day, and calculate the SPLV return from reinvesting on that day and selling at the end of the period
        etf_r = (group.loc[sell_index, 'Close'] - group.loc[group.index[0], 'Close']) / group.loc[group.index[0], 'Close']
        splv_r = (group.loc[group.index[-1], f'Close_{exit_etf}'] - group.loc[sell_index, f'Open_{exit_etf}']) / group.loc[sell_index, f'Open_{exit_etf}']

    # If it didn't trigger
    else:
        
        # Just calculate the etf return when held throughout the whole period
        etf_r = (group.loc[group.index[-1], 'Close'] - group.loc[group.index[0], 'Open']) / group.loc[group.index[0], 'Open']
        splv_r = 0

    # Calculate total return over the period
    r = (1+etf_r)*(1+splv_r)

    return pd.DataFrame({'ETF': group['Ticker'].iloc[0], 'Return': r}, index=[0])

In [13]:
def full_backtest(data:pd.DataFrame, stoploss_max:int, stoploss_accuracy:float, period_len_M=6, period_avg=False):
    exit_etf = data.loc[0, 'exit']

    # Infer if High or Low came first in the day just by which is closer to the Open/Close price
    condition = data['High'] - data['Open'] < data['High'] - data['Close']
    data.loc[condition, 'First'] = 'High'
    data.loc[~condition, 'First'] = 'Low'

    # Create the list of stoplosses to test based on parameters entered
    stoplosses = [x * stoploss_accuracy for x in list(range(int(stoploss_max*0.01/stoploss_accuracy)))]

    # Initialize Dataframe for results of backtesting
    results = pd.DataFrame()

    # For every stoploss:
    for stoploss in stoplosses:

        # Calculate the stoploss price at each day
        grouped = data.groupby('Ticker')
        data['Stoploss'] = grouped['High'].transform(lambda x: x.cummax() * (1-stoploss))

        # Determine the day we sold by seeing if price dipped below stoploss ; Use Low price if High came first, and close if high came after low
        condition = data['First'] == 'High'
        data.loc[condition, 'Sell?'] = data['Low'] < data['Stoploss']
        data.loc[~condition, 'Sell?'] = data['Close'] < data['Stoploss']

        # Calculate the total returns for each period based off the first day we would've sold
        grouped = data.groupby([pd.Grouper(key='Date', freq=f'{period_len_M}M'), 'Ticker'])
        rets = grouped.apply(total_returns, exit_etf)

        # If the period average parameter is False (default)
        if not period_avg:

            # Multiply all the period returns to get the entire timeframe return
            returns = rets.groupby('ETF').apply(lambda group: group['Return'].prod()).reset_index(name='Return')
            returns['Stoploss'] = stoploss

        elif period_avg:
            # Take the average of the period returns
            returns = rets.groupby('ETF').apply(lambda group: group['Return'].mean()).reset_index(name='Return')
            returns['Stoploss'] = stoploss

        # Add the results for that stoploss to a Dataframe
        results = pd.concat([results, returns])
    
    # Keep only the stoplosses that produced the maximum return for that etf
    best_stoplosses = results.groupby('ETF').apply(lambda group: group[group['Return'] == max(group['Return'])])
    best_stoplosses.reset_index(drop=True, inplace=True)
    return best_stoplosses

In [14]:
etfs = ['SPY', 'IJH', 'IJR', 'IEF', 'LQD', 'GSG', 'RWR', 'EFA', 'VWO']
exit_etf = 'SPLV'
start_year, end_year = 2017, 2024

data = get_OHLC_data(etfs, exit_etf, start_year, end_year)

In [15]:
# Maximum stoploss percentage to test, as well as the decimal accuracy to test
stoploss_max = 50
stoploss_accuracy = 0.01

full_backtest(data, stoploss_max, stoploss_accuracy)

,ETF,Return,Stoploss
0,EFA,1.537845,0.04
1,GSG,1.739621,0.03
2,IEF,1.593262,0.02
3,IJH,1.610252,0.45
4,IJH,1.610252,0.46
5,IJH,1.610252,0.47
6,IJH,1.610252,0.48
7,IJH,1.610252,0.49
8,IJR,1.512966,0.48
9,IJR,1.512966,0.49
